## Libraries 

In [ ]:
%pip install torch
%pip install sentence-transformers
%pip install pandas
%pip install matplotlib
%pip install --upgrade transformers sentence-transformers
%pip install pybloom_live

  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3


In [ ]:
%pip install openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install --upgrade sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 5.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1


In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np
import scipy.stats as stats
from scipy.stats import wasserstein_distance, ks_2samp
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import euclidean
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import jaccard
import os
import json
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
from scipy.sparse import csr_matrix
from openpyxl import Workbook


/Users/rebeca.mendoza/Desktop/Data_Mapper/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Data Injection (Users)

In [2]:

user1 = "Krithika"
user2 = "Yasvanth"
user3 = "Rebeca"


def load_csv_files(user):
    if user == "Krithika":
        df1 = pd.read_csv(r"C:/Users/Krithika.Patali/DataMapper/LHS_3.csv")
        df2 = pd.read_csv(r"C:/Users/Krithika.Patali/Downloads/RHS 3.csv")
        mappings_dir = r'C:\Users\Krithika.Patali\DataMapper\Mappings'
        df3 = pd.read_csv(r"C:/Users/Krithika.Patali/Downloads/synthetic_lhs_data.csv")
        df4 = pd.read_csv(r"C:/Users/Krithika.Patali/Downloads/synthetic_rhs_data.csv")
    elif user == "Yasvanth":
        df1 = pd.read_excel(r"C:\Users\Yasvanth.Pamidi\OneDrive - ENCORA\Desktop\DataMap\Trugrid\procore .xlsx")
        df2 = pd.read_excel(r"C:\Users\Yasvanth.Pamidi\OneDrive - ENCORA\Desktop\DataMap\Trugrid\salesforce_leads.xlsx")
        mappings_dir = r"C:\Users\Yasvanth.Pamidi\OneDrive - ENCORA\Desktop\VSC\DataMapper\versions\mappings"
        df3 = pd.read_csv(r"C:\Users\Yasvanth.Pamidi\OneDrive - ENCORA\Desktop\DataMap\Trugrid\synthetic_procore_data.csv")
        df4 = pd.read_csv(r"C:\Users\Yasvanth.Pamidi\OneDrive - ENCORA\Desktop\DataMap\Trugrid\synthetic_salesforce_data.csv")
    elif user == "Rebeca":
        df1 = pd.read_csv(r"/Users/rebeca.mendoza/Desktop/Data_Mapper/LHS 2.csv")
        f2 = pd.read_csv(r"/Users/rebeca.mendoza/Desktop/Data_Mapper/RHS 3.csv")
        mappings_dir = r'/Users/rebeca.mendoza/Desktop/Data_Mapper/mappings_dir'
        df3 = pd.read_csv(r"/Users/rebeca.mendoza/Desktop/Data_Mapper/synthetic_lhs_data.csv")
        df4 = pd.read_csv(r"/Users/rebeca.mendoza/Desktop/Data_Mapper/synthetic_rhs_data.csv")
        df1 = pd.read_excel(r"/Users/rebeca.mendoza/Desktop/Data_Mapper/Ahkila/procore .xlsx")
        df2 = pd.read_excel(r"/Users/rebeca.mendoza/Desktop/Data_Mapper/Ahkila/salesforce_leads.xlsx")
        mappings_dir = r'/Users/rebeca.mendoza/Desktop/Data_Mapper/mappings_dir'
        df3 = pd.read_csv(r"/Users/rebeca.mendoza/Desktop/Data_Mapper/Ahkila/synthetic_procore_data.csv")
        df4 = pd.read_csv(r"/Users/rebeca.mendoza/Desktop/Data_Mapper/Ahkila/synthetic_salesforce_data.csv")
    else:
        print("Error: User not found")
        return None

    return {"df1": df1, "df2": df2, "mappings_dir": mappings_dir, "df3": df3, "df4": df4}


files = load_csv_files(user3)

#Acces to the dataframes
if files:
    df1 = files["df1"]
    df2 = files["df2"]
    mappings_dir = files["mappings_dir"]
    df3 = files["df3"]
    df4 = files["df4"]
    print("Dataframes correctly loaded")
else:
    print("No dataframes loaded")


Dataframes correctly loaded


## Data Type functions

In [3]:
def load_mappings(mappings_dir):
    mappings = {}
    print(f"Loading mappings from directory: {mappings_dir}")

    if not os.path.exists(mappings_dir):
        raise FileNotFoundError(f"Mappings directory '{mappings_dir}' does not exist.")
    
    for filename in os.listdir(mappings_dir):
        if filename.endswith(".json") and filename != "compatibilities.json":
            database_name = os.path.splitext(filename)[0]
            print(f"Processing file: {filename} as database: {database_name}")
            with open(os.path.join(mappings_dir, filename), "r") as file:
                mappings[database_name] = json.load(file)
                print(f"Loaded mapping for {database_name}: {mappings[database_name]}")
    
    if not mappings:
        raise ValueError(f"No mapping files found in '{mappings_dir}'.")
    
    print("Mappings loaded successfully:", mappings)
    return mappings

In [4]:
def load_compatibilities(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

In [5]:
def normalize_data_type(rhs_type, mappings):
    if not mappings:
        raise ValueError("Mappings dictionary is empty or not provided.")
    
    rhs_type_lower = rhs_type.lower()
    matching_databases = []
    normalized_type = None

    for db_name, db_mapping in mappings.items():
        normalized = {k.lower(): v for k, v in db_mapping.items()}.get(rhs_type_lower)
        if normalized:
            normalized_type = normalized
            matching_databases.append(db_name)

    if matching_databases:
        return normalized_type, matching_databases
    
    return rhs_type.upper(), None

In [6]:
def are_compatible(lhs_type, rhs_type, compatibilities):
    lhs_type = lhs_type.upper()
    rhs_type = rhs_type.upper()
    return lhs_type == rhs_type or rhs_type in compatibilities.get(lhs_type, [])


## Length Functions

In [7]:
def normalize_length(length):
    """Extracts precision and scale if decimal, else returns as integer."""
    if isinstance(length, str):
        if ',' in length:  # Decimal format (e.g., "10,2")
            try:
                precision, scale = map(int, length.split(','))
                return precision, scale
            except ValueError:
                return None  # Invalid decimal format
        try:
            return int(length), None  # Normal integer/string length
        except ValueError:
            return None  # Invalid format
    return None


In [8]:
def is_length_compatible(lhs_length, rhs_length, check_length=True):
    if not check_length:
        return True

    lhs_length = normalize_length(lhs_length)
    rhs_length = normalize_length(rhs_length)

    # If either is None (invalid), return False
    if lhs_length is None or rhs_length is None:
        return False  

    # Case 1: Both are decimals (precision, scale)
    if isinstance(lhs_length, tuple) and isinstance(rhs_length, tuple):
        lhs_precision, lhs_scale = lhs_length
        rhs_precision, rhs_scale = rhs_length
        return lhs_precision <= rhs_precision and lhs_scale == rhs_scale
    
    # Case 2: One is decimal, and the other is not (Type Mismatch)
    if isinstance(lhs_length, tuple) or isinstance(rhs_length, tuple):
        return False  # Integer/String cannot be compared with Decimal

    # Case 3: General Numeric & String Length Comparison (Integer/String Lengths)
    return lhs_length <= rhs_length


In [9]:
def length_similarity_score(lhs_length, rhs_length):
    lhs_length = normalize_length(lhs_length)
    rhs_length = normalize_length(rhs_length)
    
    if lhs_length is None or rhs_length is None:
        return 0  # Return 0 if any length is invalid

    # Case 1: Both are decimal (precision, scale)
    if isinstance(lhs_length, tuple) and isinstance(rhs_length, tuple):
        lhs_precision, lhs_scale = lhs_length
        rhs_precision, rhs_scale = rhs_length
        
        # Calculate precision similarity
        precision_diff = abs(lhs_precision - rhs_precision)
        max_precision = max(lhs_precision, rhs_precision)
        precision_score = 1 - (precision_diff / max_precision) if max_precision else 1

        # Calculate scale similarity
        scale_diff = abs(lhs_scale - rhs_scale)
        max_scale = max(lhs_scale, rhs_scale)
        scale_score = 1 - (scale_diff / max_scale) if max_scale else 1

        # Average precision and scale similarity
        return (precision_score + scale_score) / 2

    # Case 2: One is decimal, the other is not (Type Mismatch)
    if isinstance(lhs_length, tuple) or isinstance(rhs_length, tuple):
        return 0  # Decimal vs Integer/String → No similarity

    # Case 3: General Numeric & String Length Comparison
    length_difference = abs(lhs_length - rhs_length)
    max_length = max(lhs_length, rhs_length)
    similarity_score = 1 - (length_difference / max_length) if max_length else 1

    return similarity_score


## Filter Compatible data using Length and Data Type 

In [10]:
type_mapping = load_mappings(mappings_dir)
compatibilities_file = os.path.join(mappings_dir, 'compatibilities.json')
compatibilities = load_compatibilities(compatibilities_file)

Loading mappings from directory: /Users/rebeca.mendoza/Desktop/Data_Mapper/mappings_dir
Processing file: saphana_sql2023_map.json as database: saphana_sql2023_map
Loaded mapping for saphana_sql2023_map: {'NVARCHAR': 'NVARCHAR', 'VARCHAR': 'VARCHAR', 'CHAR': 'CHAR', 'DATE': 'DATE', 'TIME': 'TIME', 'SECONDDATE': 'DATETIME2', 'TIMESTAMP': 'TIMESTAMP', 'DECIMAL': 'DECIMAL', 'SMALLINT': 'SMALLINT', 'INTEGER': 'INT', 'BIGINT': 'BIGINT', 'FLOAT': 'FLOAT', 'REAL': 'REAL', 'DOUBLE': 'FLOAT', 'CLOB': 'VARCHAR(max)', 'NCLOB': 'NVARCHAR(max)', 'BLOB': 'VARBINARY(max)', 'BOOLEAN': 'BIT', 'ST_GEOMETRY': 'JSON', 'ST_POINT': 'JSON', 'JSON': 'JSON', 'GUID': 'UNIQUEIDENTIFIER', 'TINYINT': 'SMALLINT'}
Processing file: sapmara_sql2023_map.json as database: sapmara_sql2023_map
Loaded mapping for sapmara_sql2023_map: {'CLNT': 'VARCHAR', 'CHAR': 'CHAR', 'DATS': 'DATE', 'UNIT': 'VARCHAR', 'NUMC': 'NUMERIC', 'QUAN': 'FLOAT', 'DEC': 'DECIMAL', 'INT2': 'SMALLINT', 'INT4': 'INT', 'CURR': 'DECIMAL', 'FLAG': 'BIT',

In [11]:
from collections import defaultdict

def filter_compatible_indices(df1, df2, type_mapping, compatibilities):
    df1['Normalized_Type'], df1['Matching_Databases'] = zip(*df1['Data_Type'].apply(lambda x: normalize_data_type(x, type_mapping)))
    df2['Normalized_Type'], df2['Matching_Databases'] = zip(*df2['Data_Type'].apply(lambda x: normalize_data_type(x, type_mapping)))
    
    compatible_index = defaultdict(list)
    length_score = defaultdict(list)
    
    for idx1, row1 in df1.iterrows():
        lhs_type = row1['Normalized_Type']
        lhs_length = row1['Length']

        for idx2, row2 in df2.iterrows():
            rhs_type = row2['Normalized_Type']
            rhs_length = row2['Length']
            
            if are_compatible(lhs_type, rhs_type, compatibilities) and is_length_compatible(lhs_length, rhs_length, check_length=True):
                score = length_similarity_score(lhs_length, rhs_length)
                compatible_index[idx1].append(idx2)
                length_score[idx1].append(score)
                
    return compatible_index, length_score

## Other Functions

In [12]:
def batch_encode(column, batch_size, model):
    embeddings = []
    column = column.tolist()
    
    with torch.no_grad():
        for i in tqdm(range(0, len(column), batch_size), desc="Encoding Batches"):
            batch = column[i:i+batch_size]
            batch_embeddings = model.encode(batch, convert_to_tensor=True, show_progress_bar=False)
            embeddings.append(batch_embeddings)
    return torch.cat(embeddings, dim=0)

In [13]:
def calculate_similarities(compatible_indices, df1, df2, column1, column2, model, calculated_similarities):
    embeddings1 = batch_encode(df1[column1], batch_size=128, model=model)
    embeddings2 = batch_encode(df2[column2], batch_size=128, model=model)

    similarity_matrix = np.zeros((len(df1), len(df2)))

    for idx1, compatible_idxs in compatible_indices.items():
        for idx2 in compatible_idxs:
            if (idx1, idx2) not in calculated_similarities:
                similarity_score = util.cos_sim(embeddings1[idx1], embeddings2[idx2]).item()
                similarity_matrix[idx1, idx2] = (similarity_score + 1) / 2  
                calculated_similarities.add((idx1, idx2))

    return csr_matrix(similarity_matrix)

In [14]:
def convert_to_native_type(value):
    if isinstance(value, (np.integer, np.int64)):
        return int(value)
    elif isinstance(value, (np.floating, np.float64)):
        return float(value)
    elif isinstance(value, np.ndarray):
        return value.tolist()
    else:
        return value

## Top similar Sentences 

In [15]:
# Function to retrieve top similar sentences based on similarity and compatibility
import heapq


def retrieve_top_similar_sentences_json(selected_index, top_n, sparse_similarity_matrix, filtered_df1, filtered_df2, column1, column2, similarity_threshold, compatible_indices, length_scores):
    # Convert the sparse similarity matrix row for the selected index to a dense array and flatten it
    similarities = sparse_similarity_matrix[selected_index].toarray().flatten()
    # Initialize a list to store matches
    matches = []

    # Check if the selected index is in the compatible indices dictionary
    if selected_index in compatible_indices:  # data type and length compatibility
        # Iterate over the compatible indices for the selected index
        for idx, length_score in zip(compatible_indices[selected_index], length_scores[selected_index]):   
            # Get the similarity score for the current index
            similarity_score = similarities[idx]
            # Skip if the similarity score is below the threshold
            if similarity_score < similarity_threshold:
                continue

            # Retrieve relevant information from filtered_df2 for the current index
            #rhs_field = filtered_df2.loc[idx, 'Attribute']
            rhs_field = filtered_df2.loc[idx, 'Attribute']
            rhs_desc = filtered_df2.loc[idx, column2]
            rhs_type = filtered_df2.loc[idx, 'Data_Type']
            rhs_length = filtered_df2.loc[idx, 'Length']
            matching_databases = filtered_df2.loc[idx, 'Matching_Databases']
            
            # Append the match information to the matches list
            matches.append({
                "rank": None,
                "similarity_score": float(similarity_score),
                "rhs_index": int(filtered_df2.index[idx]),
                "rhs_field_name": rhs_field,
                "rhs_field_desc": rhs_desc,
                "rhs_data_type": rhs_type,
                "matching_databases": matching_databases if matching_databases else ["Unknown"],
                "rhs_length": convert_to_native_type(rhs_length),
                "length_score": float(length_score),
                "compatibility": "Compatible"
            })

    # Sort the matches by similarity score in descending order and keep only the top N matches
    matches = sorted(matches, key=lambda x: (x["similarity_score"], x["length_score"]), reverse=True)[:top_n]
        
    #matches = sorted(matches, key=lambda x: x["similarity_score"], reverse=True)[:top_n]

    # Assign ranks to the matches
    for rank, match in enumerate(matches, start=1):
        match["rank"] = rank

    # If no matches are found, add a message indicating no compatible matches
    if not matches:
        matches.append({"message": "No compatible matches found."})

    # Retrieve relevant information from filtered_df1 for the selected index
    lhs_field = filtered_df1.loc[selected_index, 'Field Name']
    lhs_desc = filtered_df1.loc[selected_index, column1]
    lhs_type = filtered_df1.loc[selected_index, 'Data_Type']
    lhs_length = filtered_df1.loc[selected_index, 'Length']

    # Create the result dictionary containing the LHS field information and matches
    result = {
        "lhs_field_index": selected_index,
        "lhs_field_name": lhs_field,
        "lhs_field_description": lhs_desc,
        "lhs_field_data_type": lhs_type,
        "lhs_field_length": convert_to_native_type(lhs_length),
        "matches": matches
    }

    # Return the result wrapped in a dictionary with a "results" key
    return {"results": [result]}

In [16]:
# Load mappings and compatibilities
# Load the type mappings from the mappings directory
type_mapping = load_mappings(mappings_dir)
# Define the path to the compatibilities file
compatibilities_file = os.path.join(mappings_dir, 'compatibilities.json')
# Load the compatibilities from the file
compatibilities = load_compatibilities(compatibilities_file)

# Filter compatible data
# Filter the dataframes df1 and df2 to find compatible indices based on type and length
compatible_indices, length_score = filter_compatible_indices(df1, df2, type_mapping, compatibilities)

# Convert columns to string type
# Define the column name for descriptions in df1 and df2
column1 = 'Description'
# Convert the 'Description' column in df1 to string type
df1[column1] = df1[column1].astype(str)
column2 = 'Description'
# Convert the 'Description' column in df2 to string type
df2[column2] = df2[column2].astype(str)

# Load the model
# Load the SentenceTransformer model for encoding text data
model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')


# Calculate similarity matrix

calculated_similarities = set()

# Compute similarity matrix
# Compute the cosine similarity matrix between the embeddings of df1 and df2
sparse_similarity_matrix = calculate_similarities(compatible_indices, df1, df2, column1, column2, model, calculated_similarities)




Loading mappings from directory: /Users/rebeca.mendoza/Desktop/Data_Mapper/mappings_dir
Processing file: saphana_sql2023_map.json as database: saphana_sql2023_map
Loaded mapping for saphana_sql2023_map: {'NVARCHAR': 'NVARCHAR', 'VARCHAR': 'VARCHAR', 'CHAR': 'CHAR', 'DATE': 'DATE', 'TIME': 'TIME', 'SECONDDATE': 'DATETIME2', 'TIMESTAMP': 'TIMESTAMP', 'DECIMAL': 'DECIMAL', 'SMALLINT': 'SMALLINT', 'INTEGER': 'INT', 'BIGINT': 'BIGINT', 'FLOAT': 'FLOAT', 'REAL': 'REAL', 'DOUBLE': 'FLOAT', 'CLOB': 'VARCHAR(max)', 'NCLOB': 'NVARCHAR(max)', 'BLOB': 'VARBINARY(max)', 'BOOLEAN': 'BIT', 'ST_GEOMETRY': 'JSON', 'ST_POINT': 'JSON', 'JSON': 'JSON', 'GUID': 'UNIQUEIDENTIFIER', 'TINYINT': 'SMALLINT'}
Processing file: sapmara_sql2023_map.json as database: sapmara_sql2023_map
Loaded mapping for sapmara_sql2023_map: {'CLNT': 'VARCHAR', 'CHAR': 'CHAR', 'DATS': 'DATE', 'UNIT': 'VARCHAR', 'NUMC': 'NUMERIC', 'QUAN': 'FLOAT', 'DEC': 'DECIMAL', 'INT2': 'SMALLINT', 'INT4': 'INT', 'CURR': 'DECIMAL', 'FLAG': 'BIT',

Encoding Batches: 100%|██████████| 1/1 [00:00<00:00, 11.55it/s]


In [17]:
max_value = sparse_similarity_matrix.max()
max_index = sparse_similarity_matrix.argmax()

In [18]:
max_row, max_col = np.unravel_index(max_index, sparse_similarity_matrix.shape)
print(f"El valor máximo es {max_value} en la posición ({max_row}, {max_col})")

El valor máximo es 0.6456390917301178 en la posición (3, 6)


In [20]:
# User Input with Validations
enter_index = int(input("Enter the index from LHS to process: "))
top_n = int(input("Enter the number of top similar sentences to retrieve (N): ") or 3)
#similarity_threshold = float(input("Enter the similarity score threshold (0.0 - 1.0): "))
similarity_threshold = 0.5
# Retrieve top similar sentences

output_json = retrieve_top_similar_sentences_json(
    enter_index, top_n, sparse_similarity_matrix, df1, df2, column1, column2, similarity_threshold, compatible_indices, length_score)
print(json.dumps(output_json, indent=2))



{
  "results": [
    {
      "lhs_field_index": 3,
      "lhs_field_name": "Budget",
      "lhs_field_description": "Budget allocated for the project",
      "lhs_field_data_type": "Decimal",
      "lhs_field_length": "10,2",
      "matches": [
        {
          "rank": 1,
          "similarity_score": 0.6456390917301178,
          "rhs_index": 6,
          "rhs_field_name": "Revenue",
          "rhs_field_desc": "The total annual revenue of the account",
          "rhs_data_type": "Decimal",
          "matching_databases": [
            "saphana_sql2023_map",
            "pgsql_sql2023_map"
          ],
          "rhs_length": "15,2",
          "length_score": 0.8333333333333334,
          "compatibility": "Compatible"
        }
      ]
    }
  ]
}


In [21]:
with open("output_DataType_Length_Index_1.json", "w") as json_file:     
    json.dump(output_json, json_file, indent=4) 
    print("JSON file saved successfully!")

JSON file saved successfully!


In [22]:
df1['Data_Type'].value_counts()

Data_Type
String     5
DATE       2
Decimal    1
Name: count, dtype: int64

In [23]:
df2['Data_Type'].value_counts()

Data_Type
String     10
Date        2
Decimal     1
Integer     1
Name: count, dtype: int64

## Statistics

In [24]:
# Function to convert series to numeric values
def convert_numeric_values(series):
    try:
        # Attempt to convert the series to numeric values, coercing errors to NaN
        return pd.to_numeric(series, errors='coerce')
    except Exception as e:
        # Print an error message if conversion fails
        print(f"Conversion error: {e}")
        return series

# Function to compute statistical similarity between two columns
def compute_statistical_similarity(lhs_col, rhs_col, dtype_category):
    """
    Determines if two data distributions are statistically similar based on their type.
    """
    # Initialize similarity score to 0
    similarity_score = 0
    
    # Drop NaN values from both columns
    lhs_values = lhs_col.dropna()
    rhs_values = rhs_col.dropna()
    
    # Check if the data type category is numeric
    if dtype_category in ["INT", "INTEGER", "BIGINT", "SMALLINT", "FLOAT", "DOUBLE", "REAL", "NUMERIC", "DECIMAL"]:
        # Convert columns to numeric values
        lhs_values = convert_numeric_values(lhs_col.dropna())
        rhs_values = convert_numeric_values(rhs_col.dropna())
        
        # Compute Wasserstein distance and Kolmogorov-Smirnov statistic if both columns have values
        if len(lhs_values) > 0 and len(rhs_values) > 0:
            wasserstein_dist = stats.wasserstein_distance(lhs_values, rhs_values)
            ks_stat, ks_p_value = stats.ks_2samp(lhs_values, rhs_values)
            # Calculate similarity score based on Wasserstein distance and KS statistic
            similarity_score = 1 / (1 + wasserstein_dist) if wasserstein_dist > 0 else 1
            similarity_score = min(similarity_score, 1 - ks_stat)  # Normalize
        else:
            similarity_score = 0  # No valid comparison
    
    # Check if the data type category is categorical
    elif dtype_category in ["CHAR", "VARCHAR", "TEXT", "STRING", "JSON"]:
        # Convert columns to unique string values
        lhs_values = lhs_col.dropna().astype(str).unique()
        rhs_values = rhs_col.dropna().astype(str).unique()
        
        # Compute Jaccard similarity if both columns have values
        if len(lhs_values) > 0 and len(rhs_values) > 0:
            vectorizer = CountVectorizer(preprocessor=lambda x: x, binary=True)
            matrix = vectorizer.fit_transform([' '.join(lhs_values), ' '.join(rhs_values)])
            similarity_score = 1 - jaccard(matrix.toarray()[0], matrix.toarray()[1])
        else:
            print("something happen")
            similarity_score = 0  # No valid comparison
    elif dtype_category in ["DATE", "TIMESTAMP", "TIME", "DATETIME"]:

        # Convert columns to datetime values
        lhs_values = pd.to_datetime(lhs_col, errors='coerce')
        rhs_values = pd.to_datetime(rhs_col, errors='coerce')
        
        # Drop NaN values from both columns
        lhs_values = lhs_values.dropna()
        rhs_values = rhs_values.dropna()
        
        # Compute similarity score based on min and max date values
        if not lhs_values.empty and not rhs_values.empty:
            min_date_col1 = lhs_values.min()
            max_date_col1 = lhs_values.max()
            min_date_col2 = rhs_values.min()
            max_date_col2 = rhs_values.max()
            
            # Calculate similarity score based on date range overlap
            similarity_score = 1 - (abs((min_date_col1 - min_date_col2).days) + abs((max_date_col1 - max_date_col2).days)) / 365
        else:
            similarity_score = 0  # No valid comparison
    elif dtype_category in ["BOOLEAN"]:
        # Convert columns to boolean values
        lhs_values = lhs_col.dropna().astype(bool)
        rhs_values = rhs_col.dropna().astype(bool)
        
        # Compute similarity score based on Jaccard similarity of boolean values
        similarity_score = jaccard_score(lhs_values, rhs_values)
    else:
        print("NOT WORK FOR THIS DATA TYPE")
        similarity_score = 0  # Incompatible types
    
    return similarity_score

# Function to truncate percentage values to two decimal places
def truncate_percentage(value):
    return f"{value:.2f}"

In [26]:
# Function to retrieve top similar sentences based on similarity and statistical compatibility
def retrieve_top_similar_sentences_json(selected_index, top_n, sparse_similarity_matrix, filtered_df1, filtered_df2, column1, column2, similarity_threshold, statistical_threshold, compatible_indices, length_scores):
    # Convert the sparse similarity matrix row for the selected index to a dense array and flatten it
    similarities = sparse_similarity_matrix[selected_index].toarray().flatten()
    # Initialize a list to store matches
    matches = []

    # Check if the selected index is in the compatible indices dictionary
    if selected_index in compatible_indices:
        # Iterate over the compatible indices for the selected index
        for idx, length_score in zip(compatible_indices[selected_index], length_scores[selected_index]):  
            # Get the similarity score for the current index
            similarity_score = similarities[idx]
            # Skip if the similarity score is below the threshold
            if similarity_score < similarity_threshold:
                continue

            # Retrieve relevant information from filtered_df2 for the current index
            rhs_field = filtered_df2.loc[idx, 'Attribute']
            rhs_desc = filtered_df2.loc[idx, column2]
            rhs_type = filtered_df2.loc[idx, 'Data_Type']
            normalized_rhs_type = filtered_df2.loc[idx, 'Normalized_Type']
            rhs_length = filtered_df2.loc[idx, 'Length']
            matching_databases = filtered_df2.loc[idx, 'Matching_Databases']
            
            # Retrieve the field name from filtered_df1 for the selected index
            lhs_field = filtered_df1.loc[selected_index, 'Field Name']

            # Check if the lhs_field and rhs_field exist in df3 and df4 respectively
            if lhs_field in df3.columns and rhs_field in df4.columns:
                # Retrieve the columns from df3 and df4
                lhs_col = df3[lhs_field]
                rhs_col = df4[rhs_field]
                # Compute the statistical similarity between the columns
                stat_similarity = compute_statistical_similarity(lhs_col, rhs_col, normalized_rhs_type)
            else:
                stat_similarity = 0  # No valid comparison
                
            # Convert the statistical similarity to a percentage
            stat_similarity_percentage = truncate_percentage(stat_similarity * 100)
            
            # Skip if the statistical similarity is below the threshold
            if stat_similarity < statistical_threshold:
                continue
            
                
            # Append the match information to the matches list
            matches.append({
                "rank": None,
                "similarity_score": float(similarity_score),
                "rhs_index": int(filtered_df2.index[idx]),
                "rhs_field_name": rhs_field,
                "rhs_field_desc": rhs_desc,
                "rhs_data_type": rhs_type,
                "matching_databases": matching_databases if matching_databases else ["Unknown"],
                "rhs_length": convert_to_native_type(rhs_length),
                "length_score": float(length_score),
                "statistical_similarity": stat_similarity_percentage  
            })

    # Sort the matches by similarity score in descending order and keep only the top N matches
    #matches = sorted(matches, key=lambda x: x["similarity_score"], reverse=True)[:top_n]
    matches = sorted(matches, key=lambda x: (x["similarity_score"],x["length_score"], x["statistical_similarity"]), reverse=True)[:top_n]

    # Assign ranks to the matches
    for rank, match in enumerate(matches, start=1):
        match["rank"] = rank

    # If no matches are found, add a message indicating no compatible matches
    if not matches:
        matches.append({"message": "No compatible matches found."})

    # Retrieve relevant information from filtered_df1 for the selected index
    lhs_field = filtered_df1.loc[selected_index, 'Field Name']
    lhs_desc = filtered_df1.loc[selected_index, column1]
    lhs_type = filtered_df1.loc[selected_index, 'Data_Type']
    lhs_length = filtered_df1.loc[selected_index, 'Length']

    # Create the result dictionary containing the LHS field information and matches
    result = {
        "lhs_field_index": selected_index,
        "lhs_field_name": lhs_field,
        "lhs_field_description": lhs_desc,
        "lhs_field_data_type": lhs_type,
        "lhs_field_length": convert_to_native_type(lhs_length),
        "matches": matches
    }

    # Return the result wrapped in a dictionary with a "results" key
    return {"results": [result]}

In [29]:
# User Input with Validations
enter_index = int(input("Enter the index from LHS to process: "))
top_n = int(input("Enter the number of top similar sentences to retrieve (N): ") or 3)
#similarity_threshold = float(input("Enter the similarity score threshold (0.0 - 1.0): "))
#statistical_threshold = float(input("Enter the statistical similarity score threshold (0.0 - 1.0): "))

similarity_threshold = 0.5
statistical_threshold = 0.0

# Retrieve top similar sentences
output_json = retrieve_top_similar_sentences_json(
    enter_index, top_n, sparse_similarity_matrix, df1, df2, column1, column2, similarity_threshold,statistical_threshold, compatible_indices, length_score)
print(json.dumps(output_json, indent=2))

{
  "results": [
    {
      "lhs_field_index": 3,
      "lhs_field_name": "Budget",
      "lhs_field_description": "Budget allocated for the project",
      "lhs_field_data_type": "Decimal",
      "lhs_field_length": "10,2",
      "matches": [
        {
          "rank": 1,
          "similarity_score": 0.6456390917301178,
          "rhs_index": 6,
          "rhs_field_name": "Revenue",
          "rhs_field_desc": "The total annual revenue of the account",
          "rhs_data_type": "Decimal",
          "matching_databases": [
            "saphana_sql2023_map",
            "pgsql_sql2023_map"
          ],
          "rhs_length": "15,2",
          "length_score": 0.8333333333333334,
          "statistical_similarity": "1.20"
        }
      ]
    }
  ]
}


In [62]:
with open("output_statistics_compatibility_Index_1.json", "w") as json_file:     
    json.dump(output_json, json_file, indent=4) 
    print("JSON file saved successfully!")

JSON file saved successfully!


## Profiling 

In [30]:
import pandas as pd
import IPython.display as display


# Load data into DataFrames
lhs_df = df3
rhs_df = df4

# Define columns of interest
lhs_column = output_json["results"][0]["lhs_field_name"]
if "message" not in output_json["results"][0]["matches"][0]:
    rhs_columns = [match["rhs_field_name"] for match in output_json["results"][0]["matches"]]
else:
    rhs_columns = []

# Extract relevant columns
lhs_data = lhs_df[[lhs_column]]
rhs_data = rhs_df[rhs_columns]

# Generate summary statistics
summary_stats = pd.DataFrame(columns=["Column", "Null Values", "Distinct Values", "Non-Distinct Values", "Most Common Value", "Mean Length", "Min Length", "Max Length"])

for col in [lhs_column] + rhs_columns:
    null_values = lhs_df[col].isnull().sum() if col == lhs_column else rhs_df[col].isnull().sum()
    distinct_values = lhs_df[col].nunique() if col == lhs_column else rhs_df[col].nunique()
    total_values = len(lhs_df[col]) if col == lhs_column else len(rhs_df[col])
    non_distinct_values = total_values - distinct_values
    most_common_value = lhs_df[col].mode()[0] if col == lhs_column else rhs_df[col].mode()[0]
    
    mean_length = lhs_df[col].astype(str).str.len().mean() if col == lhs_column else rhs_df[col].astype(str).str.len().mean()
    min_length = lhs_df[col].astype(str).str.len().min() if col == lhs_column else rhs_df[col].astype(str).str.len().min()
    max_length = lhs_df[col].astype(str).str.len().max() if col == lhs_column else rhs_df[col].astype(str).str.len().max()
    
    summary_stats.loc[len(summary_stats)] = [col, null_values, distinct_values, non_distinct_values, most_common_value, mean_length, min_length, max_length]

# Display summary statistics
display.display(summary_stats)

# Generate detailed view with inferred patterns and data types
detailed_view = pd.DataFrame(columns=["Column", "Data Type", "Top Patterns", "Inferred Data Domain"])

for col in [lhs_column] + rhs_columns:
    data_type = lhs_df[col].dtype if col == lhs_column else rhs_df[col].dtype
    inferred_patterns = lhs_df[col].astype(str).str.extract(r'([A-Za-z]+)')[0].dropna().unique() if col == lhs_column else rhs_df[col].astype(str).str.extract(r'([A-Za-z]+)')[0].dropna().unique()
    inferred_data_domain = "Categorical" if data_type == 'object' else "Numerical"
    
    detailed_view.loc[len(detailed_view)] = [col, data_type, inferred_patterns, inferred_data_domain]

# Display detailed view
display.display(detailed_view)


,Column,Null Values,Distinct Values,Non-Distinct Values,Most Common Value,Mean Length,Min Length,Max Length
0,Budget,0,775,13,2749.818,7.775381,4,8
1,Revenue,0,1287,178,268.994,7.785666,5,8


,Column,Data Type,Top Patterns,Inferred Data Domain
0,Budget,float64,[],Numerical
1,Revenue,float64,[],Numerical
